In [1]:
import os
import numpy as np
import pickle
import shutil 
import pImpactR as impact
import matplotlib.pyplot as plt
import time
from copy import deepcopy as copy

# Simulation Setting

In [2]:
npt = 1024  # must be larger than 1024
Espread = 5.0e-3
Rsphere = 0.34
pipe_radius = 0.02
nTurn = 128
subTurn = 4

# Lattice

In [3]:
beam,lattice=impact.readInputFile('IOTA.Chris.in')
beam.nCore_y = 1
beam.nCore_z = 1
beam.n_particles = npt

reading ImpactZ input file (IOTA.Chris.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [4]:
lattice = [ item for item in lattice if not item.type == 'write_raw_ptcl' ]
lattice = [ item for item in lattice if not item.type == '-8' ]
lattice[0].turns = 1
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        lattice[i].n_sckick = int(np.ceil(lattice[i].length*80))
        lattice[i].n_map = 1
    if 'pipe_radius' in lattice[i].keys() and lattice[i].type!='dipole':
        lattice[i].pipe_radius = pipe_radius
    if 'dipole' == lattice[i].type:
        if lattice[i].file_id < 300:
            lattice[i].file_id = 350
            lattice[i].pipe_radius = lattice[i].pipe_radius*2
        
elemWrite = impact.getElem('write_raw_ptcl')
elemWrite.file_id   = 999
elemWrite.format_id = 2
lattice.append(elemWrite)

In [5]:
arc = lattice.copy()
nlinsert = arc.pop(1)

In [6]:
NL_nu = nlinsert.tune_advance
NL_L  = nlinsert.length
NL_c  = nlinsert.transverse_scale_c
NL_t  = nlinsert.strength_t
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
k

1.4544633270832747

In [7]:
nlinsert.n_sckick = 2
nlinsert.n_map = 50

elemTBT = impact.getElem('TBT_integral')
elemTBT.strength_t = NL_t
elemTBT.transverse_scale_c = NL_c
elemTBT.betx = betx
elemTBT.alfx = alfx
elemTBT.file_id = 10
elemTBT.pID_begin = 1
elemTBT.pID_end = npt

lattice.insert(-1,elemTBT)

initInv = copy([lattice[0],lattice[-2]])
beam.distribution.file_id = 999

In [8]:
ke = beam.kinetic_energy
freq = beam.frequency
mass = beam.mass
gam0 = 1.0+beam.kinetic_energy/mass
bet0 = np.sqrt(1.0-1.0/gam0**2)
bg0  = np.sqrt(gam0**2-1.0)
q_m  = beam.multi_charge.q_m[0]

In [9]:
def Impact2norm(data_in,bg0,bet0,sign=1):
    data=data_in.copy()
    data[:,5] = -(np.sqrt(1.0-2.0*data[:,5]/mass/(bet0*bg0)+(data[:,5]/mass)**2/bg0**2)-1.0)
    data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    return data
    
def norm2Impact(data_in,bg0,bet0,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
    data[:,5] = (bg0*np.sqrt(1/bet0**2+2.0*data[:,5]+data[:,5]**2)-bg0/bet0)*mass
    return data

In [10]:
SextIndex=[]
SextStrength=[]
for i in range(len(arc)):
    if arc[i]['type']=='multipole_thin':
        SextIndex.append(i)
        SextStrength.append(arc[i]['KL_sext'])
nParam = len(SextIndex)

In [11]:
def getLost():
    with open('lost_partcl.data') as f:
        for i, l in enumerate(f):
            pass
    return i

In [12]:
def getTBTobj(TBT,TBT0):
    obj=0.0
    for i in range(subTurn):
        obj=obj + np.sum((TBT[i,0,:]/TBT0[0,0,:]-1.0)**2) + np.sum((TBT[i,1,:]/TBT0[0,1,:]-1.0)**2)
    return np.sqrt(obj)/(npt*2)

In [13]:
pData=np.zeros([npt,9])
pData[:,6]  = q_m
pData[:,-1] = np.arange(1,npt+1)


def objFunc(arg): 
    # multipole strength regularization
    obj1 = 3.0*np.sqrt(np.sum(np.array(arg)**2)/nParam)+0.1
    if obj1 > 1.0:
        return obj1
    
    target = impact.opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = impact.opt.id_generator()
    os.mkdir(target)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(SextIndex):
        arc[j]['KL_sext']=20.0*arg[i]
    
    # prepare particles
    x = np.random.random([npt,4])
    c0 = np.cos(x[:,0])
    s0 = np.sin(x[:,0])
    c1 = np.cos(x[:,1])
    s1 = np.sin(x[:,1])
    c2 = np.cos(x[:,2])
    s2 = np.sin(x[:,2])
    pData[:,0] = Rsphere*c0
    pData[:,1] = Rsphere*s0*c1
    pData[:,2] = Rsphere*s0*s1*c2
    pData[:,3] = Rsphere*s0*s1*s2
    pData[:,5] = (x[:,3]-0.5)*Espread
    
    # arc objective
    beam.distribution.distribution_type='ReadFile'
    impact.writeParticleData(norm2Impact(pData,bg0,bet0,-1),ke,mass,freq)
    arc[0].turns=1
    arc[-1].turn=1
    impact.writeInputFile(beam,arc)
    impact.run()
    nLost = getLost()
    if 0 < nLost:
        os.chdir('..')
        shutil.rmtree(target)
        return nLost*obj1
    pData2 = impact.readParticleData(999, ke, mass, freq, format_id=2)
    pData2 = Impact2norm(pData2,bg0,bet0,1)
    obj2 = np.sqrt(np.sum((pData[:,:4]-pData2[:,:4])**2))/(4*npt*Rsphere)*20
    if obj2 > 1.0:
        os.chdir('..')
        shutil.rmtree(target)
        return obj1*obj2
    
    # first subTurns over iota ring for invariants preservation objective
    beam.n_particles = npt
    lattice[0].turns=4
    lattice[-1].turn=4
    impact.writeParticleData(norm2Impact(pData,bg0,bet0,1),ke,mass,freq)
    impact.writeInputFile(beam,lattice)
    impact.run()
    nLost = getLost()
    if 0 < nLost:
        os.chdir('..')
        shutil.rmtree(target)
        return nLost*obj1*0.8
    dummy,TBT = impact.readTBT_integral(10)
    impact.writeInputFile(beam,initInv)
    impact.run()
    dummy,TBT0 = impact.readTBT_integral(10)
    obj3 = getTBTobj(TBT,TBT0)*20
    if obj3 > 1.0:
        return obj1*obj2*obj3

    # consecutive turns over iota ring for invariants preservation objective
    beam.distribution.distribution_type='ReadFile_binary'
    impact.writeInputFile(beam,lattice)
    obj3tot=obj3
    for i in range(int(nTurn/subTurn)-1):
        impact.run()
        nLost = getLost()
        if 0 < nLost:
            os.chdir('..')
            shutil.rmtree(target)
            return nLost*obj1*(0.8**(i+1))
        dummy,TBT = impact.readTBT_integral(10)
        dummy = getTBTobj(TBT,TBT0)*20
        if dummy > 1.0:
            os.chdir('..')
            shutil.rmtree(target)
            return obj1*obj2*obj3
        obj3tot = obj3tot+dummy
        obj3 = (obj3tot/(i+2))*(0.8**(i+1))
    
    os.chdir('..')
    shutil.rmtree(target)
    return obj1*obj2*obj3

In [14]:
# objFunc([0]*nParam)

In [15]:
#%% run optim
bounds = [(-1.0,1.0)]*nParam
# result=impact.opt.differential_evolution(objFunc, bounds, ncore=32, popsize=64, 
#                                         disp=True, polish=False, maxtime=60*30) 
#                                         # stop running at maximum 1 min
# print(result)
# with open('result.sphere.cascade.12sext','wb') as fp:
#     pickle.dump(result,fp)

In [16]:
with open('result.sphere.cascade.12sext','rb') as f:
    result = pickle.load(f)

In [ ]:
while True:
    previous_result = result
    result = impact.opt.differential_evolution(objFunc, bounds, ncore=8, 
                                           prev_result=previous_result, 
                                           disp=True, polish=True, maxtime=60*30)
    with open('result.sphere.cascade.12sext','wb') as fp:
        pickle.dump(result,fp)
        if hasattr(result,'x'): 
            break

differential_evolution step 0: f(x)= 1.87512e-08
